The alternative nitrogenases V-nitrogenase and Fe-nitrogenase are less efficient that Mo-nitrogenase requiring more energy per mol of N2 reduced. This increase in energy demand while slows the observed growth rate does not slow it in proportion to the increased energy demand. With the model developed in this project we can ask a key question on how does Av reorganize the carbon metabolism and ETS to compensate for the alternative nitrogenases. 

First we will make three models for each nitrogenase and determine the flux at the condtions of 108 uM. 

In [11]:
import cobra.test
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from os.path import join
from cobra import Model, Reaction, Metabolite
from cobra.io import save_json_model
from cobra.sampling import sample
import numpy as np
import time
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
from scipy.stats import mannwhitneyu
from scipy.stats import kruskal
from cobra.sampling import OptGPSampler
from scipy.stats import wilcoxon
from numpy import std, mean, sqrt
import seaborn as sns

In [12]:
#ATPM rate at 110.7 mmmol of ATP at 108uM at 0.2 growth

model_n2_Mo = cobra.io.load_json_model("../Data/Models/iAA1300_C.json")
model_n2_V = cobra.io.load_json_model("../Data/Models/iAA1300_C.json")
model_n2_Fe = cobra.io.load_json_model("../Data/Models/iAA1300_C.json")

#make sure glucose is off
model_n2_Mo.reactions.get_by_id("EX_glc__D_e").lower_bound = 0
model_n2_Mo.reactions.get_by_id("EX_glc__D_e").upper_bound = 0
model_n2_V.reactions.get_by_id("EX_glc__D_e").lower_bound = 0
model_n2_V.reactions.get_by_id("EX_glc__D_e").upper_bound = 0
model_n2_Fe.reactions.get_by_id("EX_glc__D_e").lower_bound = 0
model_n2_Fe.reactions.get_by_id("EX_glc__D_e").upper_bound = 0

#set sucrose uptake to observed rate
model_n2_Mo.reactions.get_by_id("EX_sucr_e").lower_bound = -9
model_n2_Mo.reactions.get_by_id("EX_sucr_e").upper_bound = -9
model_n2_V.reactions.get_by_id("EX_sucr_e").lower_bound = -9
model_n2_V.reactions.get_by_id("EX_sucr_e").upper_bound = -9
model_n2_Fe.reactions.get_by_id("EX_sucr_e").lower_bound = -9
model_n2_Fe.reactions.get_by_id("EX_sucr_e").upper_bound = -9

#Set ATPM rate to predicted rate
model_n2_Mo.reactions.get_by_id("ATPM").lower_bound = 110.7
model_n2_Mo.reactions.get_by_id("ATPM").upper_bound = 110.7
model_n2_V.reactions.get_by_id("ATPM").lower_bound = 110.7
model_n2_V.reactions.get_by_id("ATPM").upper_bound = 110.7
model_n2_Fe.reactions.get_by_id("ATPM").lower_bound = 110.7
model_n2_Fe.reactions.get_by_id("ATPM").upper_bound = 110.7

#set bounds to zero for NDH I
model_n2_Mo.reactions.get_by_id("NADH6").lower_bound = 0
model_n2_Mo.reactions.get_by_id("NADH6").upper_bound = 0
model_n2_V.reactions.get_by_id("NADH6").lower_bound = 0
model_n2_V.reactions.get_by_id("NADH6").upper_bound = 0
model_n2_Fe.reactions.get_by_id("NADH6").lower_bound = 0
model_n2_Fe.reactions.get_by_id("NADH6").upper_bound = 0

#set bounds to zero for Cyt c
model_n2_Mo.reactions.get_by_id("CYOO2pp").lower_bound = 0
model_n2_Mo.reactions.get_by_id("CYOO2pp").upper_bound = 0
model_n2_V.reactions.get_by_id("CYOO2pp").lower_bound = 0
model_n2_V.reactions.get_by_id("CYOO2pp").upper_bound = 0
model_n2_Fe.reactions.get_by_id("CYOO2pp").lower_bound = 0
model_n2_Fe.reactions.get_by_id("CYOO2pp").upper_bound = 0

#make sure O2 is unbounded  
model_n2_Mo.reactions.get_by_id("O2tpp").lower_bound = 0
model_n2_Mo.reactions.get_by_id("O2tpp").upper_bound = 1000
model_n2_V.reactions.get_by_id("O2tpp").lower_bound = 0
model_n2_V.reactions.get_by_id("O2tpp").upper_bound = 1000
model_n2_Fe.reactions.get_by_id("O2tpp").lower_bound = 0
model_n2_Fe.reactions.get_by_id("O2tpp").upper_bound = 1000

#set Mo-nitrogense to zero for V-nitrogenase
model_n2_V.reactions.get_by_id("NIT1b").lower_bound = 0
model_n2_V.reactions.get_by_id("NIT1b").upper_bound = 0

#set Mo-nitrogense and v-nitrogenase to zero for fe-only-nitrogenase
model_n2_Fe.reactions.get_by_id("NIT1b").lower_bound = 0
model_n2_Fe.reactions.get_by_id("NIT1b").upper_bound = 0
model_n2_Fe.reactions.get_by_id("VNIT").lower_bound = 0
model_n2_Fe.reactions.get_by_id("VNIT").upper_bound = 0

Lets see how they grow:

In [13]:
models = [model_n2_Mo, model_n2_V, model_n2_Fe ]
models_names = ["model_n2_Mo", "model_n2_V", "model_n2_Fe" ]
for i, j in zip(models, models_names):
    model = i.copy()
    
    print("Diazotrophic growth for {} : {}".format(j, model.optimize().objective_value))
    

Diazotrophic growth for model_n2_Mo : 0.19816620149628947
Diazotrophic growth for model_n2_V : 0.17309308892841585
Diazotrophic growth for model_n2_Fe : 0.13813726684335645


As we can see the alternatives grow slightly slower than the Mo-nitrogenase showing compensation for the increased energy demands of 30 and 60%  respectively. 

Lets look at the escher maps:

In [14]:
import escher
from escher import Builder
from time import sleep

In [15]:
 Builder(model_json = "../Data/Models/iAA1300.json",
                 map_json = '../Data/Escher/Maps/Full_metabolism_AV.json',
                  reaction_data = model_n2_Mo.optimize().fluxes
                      )

Builder(reaction_data={'ALATA_D2': 0.0, 'SHCHD2': 4.4191062933666204e-05, 'CPPPGO': 4.4191062933666204e-05, 'G…

In [16]:
 Builder(model_json = "../Data/Models/iAA1300.json",
                 map_json = '../Data/Escher/Maps/Full_metabolism_AV.json',
                  reaction_data = model_n2_V.optimize().fluxes
                      )

Builder(reaction_data={'ALATA_D2': 0.0, 'SHCHD2': 3.859975883103566e-05, 'CPPPGO': 3.859975883103566e-05, 'GTH…

In [17]:
 Builder(model_json = "../Data/Models/iAA1300.json",
                 map_json = '../Data/Escher/Maps/Full_metabolism_AV.json',
                  reaction_data = model_n2_Fe.optimize().fluxes
                      )

Builder(reaction_data={'ALATA_D2': 0.0, 'SHCHD2': 3.080461050607628e-05, 'CPPPGO': 3.080461050607628e-05, 'GTH…

As we can see from the maps there is a increase in flux but no new pathways so comparing these fluxes with flux sampling should make it easier to see the differences. 

In [18]:
S_model_n2_Mo = sample(model_n2_Mo, n=10000, thinning=1000,  processes=8)

In [19]:
S_model_n2_Mo.to_csv("../Data/Sampling_alternatives/Sampling_results/model_n2_Mo_samples.csv.gz", compression="gzip")

In [20]:
S_model_n2_V = sample(model_n2_V, n=10000, thinning=1000,  processes=8)

In [21]:
S_model_n2_V.to_csv("../Data/Sampling_alternatives/Sampling_results/model_n2_V_samples.csv.gz", compression="gzip")

In [22]:
S_model_n2_Fe = sample(model_n2_Fe, n=10000, thinning=1000,  processes=8)

In [23]:
S_model_n2_Fe.to_csv("../Data/Sampling_alternatives/Sampling_results/model_n2_Fe_samples.csv.gz", compression="gzip")

In [24]:
#reupload sample files
S_model_n2_Mo = pd.read_csv("../Data/Sampling_alternatives/Sampling_results/model_n2_Mo_samples.csv.gz")
S_model_n2_V= pd.read_csv("../Data/Sampling_alternatives/Sampling_results/model_n2_V_samples.csv.gz")
S_model_n2_Fe = pd.read_csv("../Data/Sampling_alternatives/Sampling_results/model_n2_Fe_samples.csv.gz")


In [25]:
def make_svg_norm_suc(i,j,samples1,samples2,samples3,x1,x2,x3,Condition):
    #"#84a955" greenish "#bc5d41" orange #965da7 purplish
    x = samples1[i]
    y = samples2[i]
    z = samples3[i]
    
    #multiple to remove negative sign from sucrose import
    x_norm = (samples1[i]/(samples1[j]*-1))
    y_norm = (samples2[i]/(samples2[j]*-1))
    z_norm = (samples3[i]/(samples3[j]*-1))
    
    u1 = model_n2_Mo.optimize().fluxes[j]*-1
    u2 = model_n2_V.optimize().fluxes[j]*-1
    u3 = model_n2_Fe.optimize().fluxes[j]*-1
    bns = 100

    
    xn, xbins, xpatches = plt.hist(x_norm, bins=bns, alpha=0.5, color = "#84a955", lw=0)
    yn, ybins, ypatches = plt.hist(y_norm, bins=bns, alpha=0.5, color = "#bc5d41", lw=0)
    zn, zbins, zpatches = plt.hist(z_norm, bins=bns, alpha=0.5, color = "#965da7", lw=0)
        
    plt.axvline(x = (x1/u1),linewidth=2, linestyle='--', color="#84a955")
    plt.axvline(x = (x2/u2),linewidth=2, linestyle='--', color="#bc5d41")
    plt.axvline(x = (x3/u3),linewidth=2, linestyle='--', color="#965da7")

        #measure effect size of data
    
    def cohen_d(x,y):
        nx = len(x)
        ny = len(y)
        dof = nx + ny - 2
        return (mean(x) - mean(y)) / sqrt(((nx-1)*std(x, ddof=1) ** 2 + (ny-1)*std(y, ddof=1) ** 2) / dof)
    
    d_xy = cohen_d(x_norm,y_norm)
    d_xz = cohen_d(x_norm,z_norm)
    d_yz = cohen_d(y_norm,z_norm)
    
    print("Reaction {} Mo vs V".format(i), d_xy)
    print("Reaction {} Mo vs Fe".format(i), d_xz)
    print("Reaction {} V vs Fe".format(i), d_yz)
    
    plt.xticks([round(min(min(x_norm),min(y_norm),min(z_norm)),3),round(max(max(x_norm),max(y_norm),max(z_norm)),3)],fontsize=30)
    plt.yticks([],fontsize=30)
    plt.title("{}".format(i),y=0.99, fontsize = 35)
    plt.tight_layout()
    plt.savefig("../Outputs/Sampling_alternatives/{0:}/{1:}_Sample_FBA.tiff".format(Condition,i), dpi= 300, format="tiff")
    plt.close()

In [26]:
for i in [ "NADH5", "RNF", "FIX", "NIT1b", "CYTBDpp", "ATPS4rpp",
         "PDH", "GLNS", "ICL", "ICDHyr", "NAD_H2", "HYD1pp", "GLCDpp", "EDD", "HEX7",
         "HEX1", "GND", "EDD", "CS", "ICDHyr", "AKGDH", "GLUSy", "PPCK", "MDH2", "ME1"]: 
    x1 = model_n2_Mo.optimize().fluxes[i]
    x2 = model_n2_V.optimize().fluxes[i]
    x3 = model_n2_Fe.optimize().fluxes[i]
    j = "EX_sucr_e"
    make_svg_norm_suc(i, j, S_model_n2_Mo, S_model_n2_V, S_model_n2_Fe, x1, x2, x3, "Alternatives_hist_suc_norm")

Reaction NADH5 Mo vs V 0.009547020560284966
Reaction NADH5 Mo vs Fe 0.29476418256342163
Reaction NADH5 V vs Fe 0.2797401509629773
Reaction RNF Mo vs V -0.0755443093265286
Reaction RNF Mo vs Fe -0.1837508531082116
Reaction RNF V vs Fe -0.10635193197595023
Reaction FIX Mo vs V -0.9340612339651809
Reaction FIX Mo vs Fe -2.0947220385352994
Reaction FIX V vs Fe -1.2376691308203307


C:\Users\alexa\anaconda3\envs\Metabolic_models\lib\site-packages\ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in double_scalars


Reaction NIT1b Mo vs V 7.551287746024712
Reaction NIT1b Mo vs Fe 7.551287746024712
Reaction NIT1b V vs Fe nan
Reaction CYTBDpp Mo vs V -0.5650373137581557
Reaction CYTBDpp Mo vs Fe -1.3742572253324634
Reaction CYTBDpp V vs Fe -0.7996349646059117
Reaction ATPS4rpp Mo vs V -0.361232903669707
Reaction ATPS4rpp Mo vs Fe -1.0301521362818495
Reaction ATPS4rpp V vs Fe -0.6663120947139425
Reaction PDH Mo vs V 0.14314539962476544
Reaction PDH Mo vs Fe -0.02099204974139816
Reaction PDH V vs Fe -0.16950386910309742
Reaction GLNS Mo vs V 0.1497962039977869
Reaction GLNS Mo vs Fe 0.2672621074545728
Reaction GLNS V vs Fe 0.11743014406524607
Reaction ICL Mo vs V -0.44621171328491155
Reaction ICL Mo vs Fe -1.0438143049121775
Reaction ICL V vs Fe -0.5624790350126238
Reaction ICDHyr Mo vs V -0.44621171327950276
Reaction ICDHyr Mo vs Fe -1.043814304908231
Reaction ICDHyr V vs Fe -0.5624790350144948
Reaction NAD_H2 Mo vs V 0.7050903008798901
Reaction NAD_H2 Mo vs Fe 0.6532595956629302
Reaction NAD_H2 V vs

Lets do a bar plot for this too 

In [27]:
import seaborn as sns

In [28]:
def make_svg_bar_norm_suc(i,j,samples1,samples2,samples3,x1,x2,x3,Condition):
    #"#84a955" greenish "#bc5d41" orange #965da7 purplish
    x = samples1[i]
    y = samples2[i]
    z = samples3[i]
    
    #multiple to remove negative sign from sucrose import
    x_norm = (samples1[i]/(samples1[j]*-1))
    y_norm = (samples2[i]/(samples2[j]*-1))
    z_norm = (samples3[i]/(samples3[j]*-1))
    

    #create merged data frame
    df = pd.DataFrame()
    df["x"] = x_norm
    df["V"] = y_norm
    df["Fe"] = z_norm
    

    df2 = pd.melt(df)
    
    
    #measure effect size of data
    
    def cohen_d(x,y):
        nx = len(x)
        ny = len(y)
        dof = nx + ny - 2
        return (mean(x) - mean(y)) / sqrt(((nx-1)*std(x, ddof=1) ** 2 + (ny-1)*std(y, ddof=1) ** 2) / dof)
    
    d_xy = cohen_d(x_norm,y_norm)
    d_xz = cohen_d(x_norm,z_norm)
    d_yz = cohen_d(y_norm,z_norm)
    
    print("Reaction {} Mo vs V".format(i), d_xy)
    print("Reaction {} Mo vs Fe".format(i), d_xz)
    print("Reaction {} V vs Fe".format(i), d_yz)
    
    plt.figure(figsize=(2, 5))
    sns.boxplot(x="variable", y="value", data=df2, showfliers=False, width = 0.25, linewidth= 1.5)
    plt.title("{}".format(i),y=0.99, fontsize = 20)
    plt.xlabel('Nitrogenase')
    plt.ylabel('Relative Flux')
    plt.tight_layout()
    plt.savefig("../Outputs/Sampling_alternatives/{0:}/{1:}_Sample_FBA.tiff".format(Condition,i), dpi= 300, format="tiff")
    plt.close()

In [29]:
for i in [ "NADH5", "RNF", "FIX", "NIT1b", "CYTBDpp", "ATPS4rpp",
         "PDH", "GLNS", "ICL", "ICDHyr", "NAD_H2", "HYD1pp", "GLCDpp", "EDD", "HEX7",
         "HEX1", "GND", "EDD", "CS", "ICDHyr", "AKGDH", "GLUSy", "PPCK", "MDH2", "ME1"]: 
    x1 = model_n2_Mo.optimize().fluxes[i]
    x2 = model_n2_V.optimize().fluxes[i]
    x3 = model_n2_Fe.optimize().fluxes[i]
    j = "EX_sucr_e"
    make_svg_bar_norm_suc(i, j, S_model_n2_Mo, S_model_n2_V, S_model_n2_Fe, x1, x2, x3, "Alternatives_bar_suc_norm")

Reaction NADH5 Mo vs V 0.009547020560284966
Reaction NADH5 Mo vs Fe 0.29476418256342163
Reaction NADH5 V vs Fe 0.2797401509629773
Reaction RNF Mo vs V -0.0755443093265286
Reaction RNF Mo vs Fe -0.1837508531082116
Reaction RNF V vs Fe -0.10635193197595023
Reaction FIX Mo vs V -0.9340612339651809
Reaction FIX Mo vs Fe -2.0947220385352994
Reaction FIX V vs Fe -1.2376691308203307


C:\Users\alexa\anaconda3\envs\Metabolic_models\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in double_scalars


Reaction NIT1b Mo vs V 7.551287746024712
Reaction NIT1b Mo vs Fe 7.551287746024712
Reaction NIT1b V vs Fe nan
Reaction CYTBDpp Mo vs V -0.5650373137581557
Reaction CYTBDpp Mo vs Fe -1.3742572253324634
Reaction CYTBDpp V vs Fe -0.7996349646059117
Reaction ATPS4rpp Mo vs V -0.361232903669707
Reaction ATPS4rpp Mo vs Fe -1.0301521362818495
Reaction ATPS4rpp V vs Fe -0.6663120947139425
Reaction PDH Mo vs V 0.14314539962476544
Reaction PDH Mo vs Fe -0.02099204974139816
Reaction PDH V vs Fe -0.16950386910309742
Reaction GLNS Mo vs V 0.1497962039977869
Reaction GLNS Mo vs Fe 0.2672621074545728
Reaction GLNS V vs Fe 0.11743014406524607
Reaction ICL Mo vs V -0.44621171328491155
Reaction ICL Mo vs Fe -1.0438143049121775
Reaction ICL V vs Fe -0.5624790350126238
Reaction ICDHyr Mo vs V -0.44621171327950276
Reaction ICDHyr Mo vs Fe -1.043814304908231
Reaction ICDHyr V vs Fe -0.5624790350144948
Reaction NAD_H2 Mo vs V 0.7050903008798901
Reaction NAD_H2 Mo vs Fe 0.6532595956629302
Reaction NAD_H2 V vs

In [30]:
def make_svg_bar_norm_suc_Mo(i,j,samples1,samples2,samples3,x1,x2,x3,Condition):
    #"#84a955" greenish "#bc5d41" orange #965da7 purplish
    x = samples1[i]
    y = samples2[i]
    z = samples3[i]
    
    #multiple to remove negative sign from sucrose import
    x_norm = (samples1[i]/(samples1[j]*-1))
    y_norm = (samples2[i]/(samples2[j]*-1))
    z_norm = (samples3[i]/(samples3[j]*-1))
    
    
    #make data relative to Mo conditions for bar plot scaling
    y_rel = y_norm/x_norm
    z_rel = z_norm/x_norm
    
    #create merged data frame
    df = pd.DataFrame()
    df["V"] = y_rel
    df["Fe"] = z_rel
    

    df2 = pd.melt(df)
    
    
    #measure effect size of data
    
    def cohen_d(x,y):
        nx = len(x)
        ny = len(y)
        dof = nx + ny - 2
        return (mean(x) - mean(y)) / sqrt(((nx-1)*std(x, ddof=1) ** 2 + (ny-1)*std(y, ddof=1) ** 2) / dof)
    
    d_xy = cohen_d(x_norm,y_norm)
    d_xz = cohen_d(x_norm,z_norm)
    d_yz = cohen_d(y_norm,z_norm)
    
    print("Reaction {} Mo vs V".format(i), d_xy)
    print("Reaction {} Mo vs Fe".format(i), d_xz)
    print("Reaction {} V vs Fe".format(i), d_yz)
    
    plt.figure(figsize=(2, 5))
    sns.boxplot(x="variable", y="value", data=df2, showfliers=False, width = 0.25, linewidth= 1.5)
    plt.axhline(y=1, color='black', linestyle='--')
    plt.annotate("Mo", xy = (1.2, 1), xycoords='data',
            xytext=(0, 10), textcoords='offset pixels' )
    plt.yticks([],fontsize=30)
    plt.title("{}".format(i),y=0.99, fontsize = 20)
    plt.xlabel('Nitrogenase')
    plt.ylabel('Relative Flux')
    plt.tight_layout()
    plt.savefig("../Outputs/Sampling_alternatives/{0:}/{1:}_Sample_FBA.tiff".format(Condition,i), dpi= 300, format="tiff")
    plt.close()

In [31]:
for i in [ "NADH5", "RNF", "FIX", "NIT1b", "CYTBDpp", "ATPS4rpp",
         "PDH", "GLNS", "ICL", "ICDHyr", "NAD_H2", "HYD1pp", "GLCDpp", "EDD", "HEX7",
         "HEX1", "GND", "EDD", "CS", "ICDHyr", "AKGDH", "GLUSy", "PPCK", "MDH2", "ME1"]: 
    x1 = model_n2_Mo.optimize().fluxes[i]
    x2 = model_n2_V.optimize().fluxes[i]
    x3 = model_n2_Fe.optimize().fluxes[i]
    j = "EX_sucr_e"
    make_svg_bar_norm_suc_Mo(i, j, S_model_n2_Mo, S_model_n2_V, S_model_n2_Fe, x1, x2, x3, "Alternatives_bar_suc_norm_Mo")

Reaction NADH5 Mo vs V 0.009547020560284966
Reaction NADH5 Mo vs Fe 0.29476418256342163
Reaction NADH5 V vs Fe 0.2797401509629773
Reaction RNF Mo vs V -0.0755443093265286
Reaction RNF Mo vs Fe -0.1837508531082116
Reaction RNF V vs Fe -0.10635193197595023
Reaction FIX Mo vs V -0.9340612339651809
Reaction FIX Mo vs Fe -2.0947220385352994
Reaction FIX V vs Fe -1.2376691308203307


C:\Users\alexa\anaconda3\envs\Metabolic_models\lib\site-packages\ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in double_scalars


Reaction NIT1b Mo vs V 7.551287746024712
Reaction NIT1b Mo vs Fe 7.551287746024712
Reaction NIT1b V vs Fe nan
Reaction CYTBDpp Mo vs V -0.5650373137581557
Reaction CYTBDpp Mo vs Fe -1.3742572253324634
Reaction CYTBDpp V vs Fe -0.7996349646059117
Reaction ATPS4rpp Mo vs V -0.361232903669707
Reaction ATPS4rpp Mo vs Fe -1.0301521362818495
Reaction ATPS4rpp V vs Fe -0.6663120947139425
Reaction PDH Mo vs V 0.14314539962476544
Reaction PDH Mo vs Fe -0.02099204974139816
Reaction PDH V vs Fe -0.16950386910309742
Reaction GLNS Mo vs V 0.1497962039977869
Reaction GLNS Mo vs Fe 0.2672621074545728
Reaction GLNS V vs Fe 0.11743014406524607
Reaction ICL Mo vs V -0.44621171328491155
Reaction ICL Mo vs Fe -1.0438143049121775
Reaction ICL V vs Fe -0.5624790350126238
Reaction ICDHyr Mo vs V -0.44621171327950276
Reaction ICDHyr Mo vs Fe -1.043814304908231
Reaction ICDHyr V vs Fe -0.5624790350144948
Reaction NAD_H2 Mo vs V 0.7050903008798901
Reaction NAD_H2 Mo vs Fe 0.6532595956629302
Reaction NAD_H2 V vs

In [32]:
for i in [ "NADH5", "RNF", "FIX", "NIT1b", "CYTBDpp", "ATPS4rpp",
         "PDH", "GLNS", "ICL", "ICDHyr", "NAD_H2", "HYD1pp", "GLCDpp", "EDD", "HEX7",
         "HEX1", "GND", "EDD", "CS", "ICDHyr", "AKGDH", "GLUSy", "PPCK", "MDH2", "ME1"]: 
    x1 = model_n2_Mo.optimize().fluxes[i]
    x2 = model_n2_V.optimize().fluxes[i]
    x3 = model_n2_Fe.optimize().fluxes[i]
    j = "O2tpp"
    make_svg_bar_norm_suc_Mo(i, j, S_model_n2_Mo, S_model_n2_V, S_model_n2_Fe, x1, x2, x3, "Alternatives_bar_o2_norm_Mo")

Reaction NADH5 Mo vs V -0.39302966570861775
Reaction NADH5 Mo vs Fe -1.2067550266272666
Reaction NADH5 V vs Fe -0.8106535844034981
Reaction RNF Mo vs V 0.07198032209310612
Reaction RNF Mo vs Fe 0.17546236848201072
Reaction RNF V vs Fe 0.10155966567307592
Reaction FIX Mo vs V 0.9059366219723239
Reaction FIX Mo vs Fe 2.0336526591045874
Reaction FIX V vs Fe 1.2008316690341154


C:\Users\alexa\anaconda3\envs\Metabolic_models\lib\site-packages\ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in double_scalars


Reaction NIT1b Mo vs V -7.483120034470791
Reaction NIT1b Mo vs Fe -7.483120034470791
Reaction NIT1b V vs Fe nan
Reaction CYTBDpp Mo vs V -0.02491103869472548
Reaction CYTBDpp Mo vs Fe -0.05304497202025238
Reaction CYTBDpp V vs Fe -0.02805632340048806
Reaction ATPS4rpp Mo vs V 0.025223190995819895
Reaction ATPS4rpp Mo vs Fe 0.24896262038718395
Reaction ATPS4rpp V vs Fe 0.22317000015432148
Reaction PDH Mo vs V -0.148128741005896
Reaction PDH Mo vs Fe 0.008142913400664854
Reaction PDH V vs Fe 0.16217895109933084
Reaction GLNS Mo vs V -0.17108144340249204
Reaction GLNS Mo vs Fe -0.31700264473040946
Reaction GLNS V vs Fe -0.14561490747691652
Reaction ICL Mo vs V 0.022291250577556824
Reaction ICL Mo vs Fe -0.021858236886836732
Reaction ICL V vs Fe -0.0432966788801585
Reaction ICDHyr Mo vs V 0.022291250570851746
Reaction ICDHyr Mo vs Fe -0.021858236891785128
Reaction ICDHyr V vs Fe -0.04329667887797352
Reaction NAD_H2 Mo vs V -0.7015477437083047
Reaction NAD_H2 Mo vs Fe -0.6455025304391518
Re

In [33]:
def make_svg_bar_nit(j,samples1,samples2,samples3,x1,x2,x3,Condition):   
    #"#84a955" greenish "#bc5d41" orange #965da7 purplish
    x = samples1["NIT1b"]
    y = samples2["VNIT"]
    z = samples3["FENIT"]
    
    #normalize to O2 import
    x_norm = (samples1["NIT1b"]/(samples1["O2tpp"]))
    y_norm = (samples2["VNIT"]/(samples2["O2tpp"]))
    z_norm = (samples3["FENIT"]/(samples3["O2tpp"]))
    
    
    #make data relative to Mo conditions for bar plot scaling
    y_rel = y_norm/x_norm
    z_rel = z_norm/x_norm
    
    #create merged data frame
    df = pd.DataFrame()
    #df["x"] = x_norm
    df["V"] = y_rel
    df["Fe"] = z_rel
    
    #print(df.dtypes)
    #df = df.astype("Int64")
    #df2 = df2.apply(pd.to_numeric) 
    #melt df 
    df2 = pd.melt(df)
    
    
    #measure effect size of data
    
    def cohen_d(x,y):
        nx = len(x)
        ny = len(y)
        dof = nx + ny - 2
        return (mean(x) - mean(y)) / sqrt(((nx-1)*std(x, ddof=1) ** 2 + (ny-1)*std(y, ddof=1) ** 2) / dof)
    
    d_xy = cohen_d(x_norm,y_norm)
    d_xz = cohen_d(x_norm,z_norm)
    d_yz = cohen_d(y_norm,z_norm)
    
    print("Reaction {} Mo vs V".format(i), d_xy)
    print("Reaction {} Mo vs Fe".format(i), d_xz)
    print("Reaction {} V vs Fe".format(i), d_yz)
    
    plt.figure(figsize=(2, 5))
    sns.boxplot(x="variable", y="value", data=df2, showfliers=False, width = 0.25, linewidth= 1.5)
    plt.axhline(y=1, color='black', linestyle='--')
    #plt.text(1.2, 1+(std(z_norm)/2), "Mo" )
    plt.annotate("Mo", xy = (1.2, 1), xycoords='data',
            xytext=(0, 10), textcoords='offset pixels' )
    plt.yticks([],fontsize=30)
    plt.title("Nitrogenase",y=0.99, fontsize = 20)
    plt.xlabel('Nitrogenase')
    plt.ylabel('Relative Flux')
    plt.tight_layout()
    plt.savefig("../Outputs/Sampling_alternatives/{0:}_nit_Sample_FBA.tiff".format(Condition), dpi= 300, format="tiff")
    plt.close()

In [34]:
make_svg_bar_nit( j, S_model_n2_Mo, S_model_n2_V, S_model_n2_Fe, x1,x2,x3,"alternatives")

Reaction ME1 Mo vs V 0.720582228442576
Reaction ME1 Mo vs Fe 1.9820904021805792
Reaction ME1 V vs Fe 1.6235493563121082
